In [123]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import csv
import time

In [124]:
all_data = []

In [ ]:
#read in blob dataset

X_training = []
y_training = []
header = []
c = 0
with open("DataSets/blob_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([ float(x) for x in row])
        
with open("DataSets/blob_label.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        y_training.append(int(row[0]))


        
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])

In [126]:
#read in iris dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/iris.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row[:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row[:-1]])
        y_training.append(row[-1])

for i in range(len(y_training)):
    if(y_training[i] == "Setosa"):
        y_training[i] = 1
    if(y_training[i] == "Versicolor"):
        y_training[i] = 2
    if(y_training[i] == "Virginica"):
        y_training[i] = 3
        
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])


In [ ]:
#read in wine dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/winequality-white_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row[:-1]])
        y_training.append(float(row[-1]))
        #print(row)
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])
print(header)


In [ ]:
#read in house dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/house_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row])
        
with open("DataSets/house_price.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        #print(row)
        y_training.append(float(row[0]))#row[0]!!!

        
print(len(X_training))
print(len(y_training))
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])


In [129]:
#Sort dataset by an attribute.(increasing order)
def sort_f(X,y,column):
    

  
    sortf = zip(X,y)
    sortf= sorted(sortf,key = lambda x:x[0][column])
    X = [x for x,y in sortf]
    y = [y for x,y in sortf]
    
    

    return X,y

In [131]:
#Sort dataset by an attribute.(increasing order)
def sort_f2(X,y,column):
    

    X = np.array(X)
    y = np.array(y)
    p = X[:,column].argsort()
    X = X[p]
    y = y[p]

    return X.tolist(),y.tolist()

In [132]:
class Tree:
    def __init__(self,X,feature_names,labels):
        
        self.X = X
        self.num_of_nodes = 0
        self.currentsplit = 0
        self.split_result = 0
        self.feature_names = feature_names #coloum names
        self.labels = labels#y
        self.catagories = set(labels)
        self.nodes = []
        self.leaf = 0
        

In [133]:
class Node:
    def __init__(self,labels,X,feature_ids):
        
        self.split_result = 0
        self.split = 0 #which column / which feature id
        self.feature_ids = feature_ids
        self.labels = labels
        self.X = X
        self.nodes = []
        self.regr = 0
        self.depth = 0
        
        self.leaf = 0 #true or false

In [134]:
def get_features(X,labels,column,split_value):
    
    #Split the dataset into two parts
    #The first part has values  lower than split_value
    #The second part has values  bigger or equal than split_value
    
    features = [ X[i][column] for i in range(len(X)) ]
    
    sub_labels = []
    
    sub_label1 = []
    sub_label2 = []
    sub_x1 = []
    sub_x2 = []
    
    for x in range(len(features)):
        if(features[x]<split_value):
            sub_label1.append(labels[x])
            sub_x1.append(X[x])
        else:
            sub_label2.append(labels[x])
            sub_x2.append(X[x])
    
    sub_labels.append([sub_label1,sub_x1,column])
    sub_labels.append([sub_label2,sub_x2,column])

    
    #return splited data,containing the labels,features and the the column    
    return sub_labels

In [135]:
def regression(regr_type,X,y,column):
    
    #calculate linear regression
    
    X = [ [X[i][column]] for i in range(len(X)) ]
    
    regr = LinearRegression()
    regr.fit(X,y)
    
    
    return regr

In [136]:
def calculate_error(regr,X,y,column):
    
    
    X = [ [X[i][column]] for i in range(len(X)) ]
    predict = regr.predict(X)
    
    difference = []
    
    for i in range(len(predict)):
        difference.append( (predict[i]-y[i])**2 )
    error = sum(difference)/len(difference)

  
    return error

In [137]:
def calculate_split_regression(X,labels,column,split_value):
    
    error = 0
    
    sub_labels = get_features(X,labels,column,split_value) 
    
    for i in sub_labels:

        if(len(i[0]) == 0):
            return -1
        #calculate linear regression for each part of the splited data
        regr = regression(1,i[1],i[0],column)
        #calculate the rror of linear regression
        error+= calculate_error(regr,i[1],i[0],column)

  
    return error

In [138]:
def find_best_split(X,labels,feature_ids):
    
    #for each column we try to find the best split_value
    #we return the split_value wich results in the lowest standard deviation
    
    lowest_error = -1
    split = 0
    split_value = 0
    V = 0 
    
    features = 0 
    svalue = 0

    for i in feature_ids:
        
            #sort
            X,labels = sort_f(X,labels,i)
            features = [ X[j][i] for j in range(len(X)) ] 
            

            #iterate through features, take two values avarage, calculate standard deviation after split
            for k in range(0,len(features)-1,2): 

                if(features[k]==features[k+1]):
                    continue
                
                svalue = (features[k]+features[k+1])/2
                V = calculate_split_regression(X,labels,i,svalue)
                
                if(lowest_error == -1):
                    lowest_error = V
                    split_value = svalue
                    split = i
                
                #after the split, if the standard deviation is lower than in a previous split, we choose this one
                if(V<lowest_error and V != -1):

                    split_value = svalue
                    lowest_error = V
                    split = i

    
    return split,split_value

In [139]:
def build_tree(X,feature_ids,labels,leaf_size,limit,depth):
    
    #find the best split for sub_nodes

    split = 0 
    sub_nodes = []
    
    #get the best split value and column
    split,split_value = find_best_split(X,labels,feature_ids)

    node_labels = get_features(X,labels,split,split_value)  

    
    #create objects and inicialize them
    for i in node_labels:
        if(len(i[0]) == 0):
            continue
        node = Node(i[0],i[1],feature_ids)
        node.split = split
        node.split_result = split_value
        node.depth = depth

        node.regr = regression(1,i[1],i[0],split)
        
        

        if(len(node.labels) <= leaf_size or calculate_error(node.regr,node.X,node.labels,split)<limit):
            node.leaf = 1

            
        else:
            node.leaf = 0
        sub_nodes.append(node)
    

    if(depth == 3000):
        for i in sub_nodes:
            i.leaf = 1
        return sub_nodes
    
    depth +=1
    
    leaf =  0
    for i in sub_nodes:
        if(i.leaf == 1):
            leaf +=1
    if(leaf == len(sub_nodes)):
   
        return sub_nodes
    
    else:
        for node in sub_nodes:

            if(node.leaf == 0):
                
                node.nodes = build_tree(node.X,node.feature_ids,node.labels,leaf_size,limit,depth)#node.depth


    return sub_nodes   

In [140]:
def inicialize(X,feature_names,labels,leaf_size,limit,div):
    
    depth = 1
    feature_ids = [x for x in range(len(feature_names))]
    tree = Tree(X,feature_names,labels)
    #start building tree
    tree.nodes = build_tree(X,feature_ids,labels,leaf_size,limit,depth)
    return tree

In [141]:
forest = []
times = []
#limits = [0.5,0.75,1,1.25,1.5,1.75,2]
#divide = [2]
size = 20
counter = 0
var = 0
#for each dataset we build a tree and mesure the elapsed time
for i in all_data: 
    f = []
    t = []
    print("-------------------------------------------------------")

    if(counter == 0):
        size =20
        var = 0.5
    if(counter == 1):
        size =10
        var = 0.2
    if(counter == 2):
        size =20
        var = 0.8
    if(counter == 3):
        size = 20
        var = 1

    start = time.time()
    tree = inicialize(i[0],i[-1],i[2],size,var,1)
    end = time.time()
    f.append(tree)
    t.append(end-start)
    counter+=1
    print(end-start)

    forest.append(f)
    times.append(t)

-------------------------------------------------------


129.25556015968323
-------------------------------------------------------
0.02221989631652832
-------------------------------------------------------
3.78776478767395
-------------------------------------------------------
134.10051584243774


In [142]:

def predict(node,value):

    if(node.leaf == 1):
        p = node.regr.predict(np.array(value[node.split]).reshape((1,-1)) )
        
        return p
    else:
        if (node.nodes[0].split_result<value[node.nodes[0].split]):
            
            p=predict(node.nodes[1],value)
            return p
        else:
            p=predict(node.nodes[0],value)
            return p
        


In [143]:


predicted_value = []#predicted points
difference = []
MSE = []
sum_y = []


all_mse = []

#for each tree we calculate the mean squared error
#all_data = [X_train,X_test,y_train,y_test,header]

for v in range(len(forest)):
    MSE = []
    for i in forest[v]:
        j = all_data[v]
        
        X_test = j[1]
        y_test = j[3]
        pv = []
        error = 0
        mse = 0
        for k in range(len(X_test)): 

            p = predict(i,X_test[k])
            #print(p,y_test[k])
           

            error += abs(y_test[k]-p)
            mse += (y_test[k]-p)**2
            pv.append(p)
            

        sum_y.append(sum(y_test))
        predicted_value.append(pv)
        difference.append(error)
        mse = mse/len(X_test)

    all_mse.append(mse)
    
    
print(difference)
print(sum_y)

print(all_mse)


with open("Results/difference_rtl.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for row in all_mse:
        writer.writerow(row)

with open("Results/times_rtl.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for row in times:
        writer.writerow(row)


[array([715.81314213]), array([3.92454552]), array([604.73511686]), array([9314.43601119])]
[4053, 58, 5793.0, 628285.3781633918]
[array([0.28037413]), array([0.03160336]), array([0.62034028]), array([44.05206218])]
